In [78]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt

import pytz

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import get_sun, get_body, get_moon
from astroplan import Observer, FixedTarget, time_grid_from_range, observability_table, moon_illumination
from astroplan import AirmassConstraint, MoonSeparationConstraint

In [79]:
mro = Observer.at_site('mro')
mro

<Observer: name='mro',
    location (lon, lat, el)=(-120.7278 deg, 46.952799999999996 deg, 1197.9999999998224 m),
    timezone=<UTC>>

In [80]:
reference_time = Time('2018-7-15')

# First: moon stuff

In [81]:
print('On the night of the 15th, the moon phase will be {0:.2f}, meaning {1:.2f}% illumination. Hell yeah.'.format(
      mro.moon_phase(reference_time), mro.moon_illumination(reference_time)*100))

On the night of the 15th, the moon phase will be 2.67 rad, meaning 5.35% illumination. Hell yeah.


# Now on to the actual observation planning (which looks good overall; definitely better than the last time I tried this, in late April):

In [82]:
tz = mro.timezone

In [83]:
constraints = [MoonSeparationConstraint(30*u.deg)]
constraints.append(AirmassConstraint(2))

In [84]:
astro_set1 = mro.twilight_evening_astronomical(reference_time, which='nearest')
astro_rise1 = mro.twilight_morning_astronomical(reference_time, which='next')
observing_length1 = (astro_rise1 - astro_set1).to(u.h)

print("Astronomical Evening Twilight starts at {0.iso} UTC".format(astro_set1))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(astro_rise1))
print("You can observe for {0:.1f} during the observing window.".format(observing_length1))

# aw flux

Astronomical Evening Twilight starts at 2018-07-15 06:30:15.548 UTC
Astronomical Morning Twilight starts at 2018-07-15 09:47:49.916 UTC
You can observe for 3.3 h during the observing window.


In [85]:
obs_range = [astro_set1, astro_rise1]

In [86]:
northern_LBV_targets = [['VI Cyg 12', 20.5447, 41.2415],
['GRS 79.29+0.46', 20.5284, 40.366425], 
['MWC 314', 19.359436, 14.88246944],
['[OMN2000] LS1', 19.3966, 14.6107],
['2MASS J19325284+1742303', 19.548014, 17.708425], 
['2MASS J19443759+2419058c', 19.74378, 24.3182972],
['2MASS J19011669+0355108', 19.021303, 3.91967], 
['MN101', 19.10682, 8.3671],
['MN107', 19.40093, 13.66372],
['P Cyg', 20.29644, 38.0329306],
['AFGL 2298', 19.003025, 3.763083]]

In [87]:
ntargets = [FixedTarget(coord=SkyCoord(ra = RA*u.hourangle, dec = DEC*u.deg), name=Name)
           for Name, RA, DEC in northern_LBV_targets]

In [88]:
observing_table = observability_table(constraints, mro, ntargets, time_range=obs_range)

print(observing_table)

      target name        ever observable ... fraction of time observable
------------------------ --------------- ... ---------------------------
               VI Cyg 12            True ...                         1.0
          GRS 79.29+0.46            True ...                         1.0
                 MWC 314            True ...                         1.0
           [OMN2000] LS1            True ...                         1.0
 2MASS J19325284+1742303            True ...                         1.0
2MASS J19443759+2419058c            True ...                         1.0
 2MASS J19011669+0355108            True ...                         1.0
                   MN101            True ...                         1.0
                   MN107            True ...                         1.0
                   P Cyg            True ...                         1.0
               AFGL 2298            True ...                         1.0


In [89]:
# right on

In [90]:
for i, target_table in enumerate(ntargets):

    if observing_table['ever observable'][i]:
        name = observing_table['target name'][i]
        obj_frac = observing_table['fraction of time observable'][i]
        obj_time = obj_frac * observing_length1
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)

You can observe VI Cyg 12 for 3.29 h
You can observe GRS 79.29+0.46 for 3.29 h
You can observe MWC 314 for 3.29 h
You can observe [OMN2000] LS1 for 3.29 h
You can observe 2MASS J19325284+1742303 for 3.29 h
You can observe 2MASS J19443759+2419058c for 3.29 h
You can observe 2MASS J19011669+0355108 for 3.29 h
You can observe MN101 for 3.29 h
You can observe MN107 for 3.29 h
You can observe P Cyg for 3.29 h
You can observe AFGL 2298 for 3.29 h


# Since the northern dec. ones definitely work out on that night, try and throw in some of the southern ones:

In [91]:
southern_LBV_targets = [
    ['GAL 024.73+00.69', 18+(33/60)+(55.29/3600), -6+(58/60)+(38.7/3600)],
    ['HD 168625', 18+(21/60)+(19.55/3600), -16+(22/60)+(26.06/3600)],
    ['GRS 25.5+0.2', 18+(37/60)+(5.21/3600), -6+(29/60)+(38/3600)],
    ['GAL 026.47+00.02', 18+(39/60)+(32.24/3600), -5+(44/60)+(20.5/3600)],
    ['2MASS J18022233-2238002', 18+2/60+22.34/3600, -22+38/60+00.24/3600],
    ['2MASS J18455593-0308297', 18+45/60+55.94/3600, -3+8/60+29.72/3600],
    ['2MASS J18510295-0058242', 18+51/60+2.95/3600, -0+58/60+24.21/3600],
    ['MN83', 18+39/3600+23.01/3600, -5+53/60+19.9/3600],
]

In [92]:
stargets = [FixedTarget(coord=SkyCoord(ra = RA*u.hourangle, dec = DEC*u.deg), name=Name)
           for Name, RA, DEC in southern_LBV_targets]

In [93]:
observing_table_s1 = observability_table(constraints, mro, stargets, time_range=obs_range)

print(observing_table_s1)

      target name       ever observable ... fraction of time observable
----------------------- --------------- ... ---------------------------
       GAL 024.73+00.69            True ...          0.8571428571428571
              HD 168625           False ...                         0.0
           GRS 25.5+0.2            True ...          0.8571428571428571
       GAL 026.47+00.02            True ...                         1.0
2MASS J18022233-2238002           False ...                         0.0
2MASS J18455593-0308297            True ...                         1.0
2MASS J18510295-0058242            True ...                         1.0
                   MN83            True ...          0.7142857142857143


In [94]:
for i, target_table in enumerate(stargets):

    if observing_table_s1['ever observable'][i]:
        name = observing_table_s1['target name'][i]
        obj_frac = observing_table_s1['fraction of time observable'][i]
        obj_time = obj_frac * observing_length1
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)

You can observe GAL 024.73+00.69 for 2.82 h
You can observe GRS 25.5+0.2 for 2.82 h
You can observe GAL 026.47+00.02 for 3.29 h
You can observe 2MASS J18455593-0308297 for 3.29 h
You can observe 2MASS J18510295-0058242 for 3.29 h
You can observe MN83 for 2.35 h


In [95]:
# note: viable targets that are confirmed LBVs:
#
# P Cyg,
# AFGL 2298,
# GAL 024.73+00.69

# For the second weekend:

In [96]:
second_reference_time = Time('2018-8-2')

# Making sure that lucky old moon doesn't interfere again:

In [97]:
print('On the night of the 15th, the moon phase will be {0:.2f}, meaning {1:.2f}% illumination. Okay thats not great.'
      .format(
      mro.moon_phase(second_reference_time), mro.moon_illumination(second_reference_time)*100))

On the night of the 15th, the moon phase will be 1.00 rad, meaning 77.17% illumination. Okay thats not great.


In [98]:
constraints2 = [MoonSeparationConstraint(40*u.deg)]
constraints2.append(AirmassConstraint(2))

In [99]:
astro_set2 = mro.twilight_evening_astronomical(second_reference_time, which='nearest')
astro_rise2 = mro.twilight_morning_astronomical(second_reference_time, which='next')
observing_length2 = (astro_rise2 - astro_set2).to(u.h)

print("Astronomical Evening Twilight starts at {0.iso} UTC".format(astro_set2))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(astro_rise2))
print("You can observe for {0:.1f} during the observing window.".format(observing_length2))

Astronomical Evening Twilight starts at 2018-08-02 05:48:52.324 UTC
Astronomical Morning Twilight starts at 2018-08-02 10:29:56.820 UTC
You can observe for 4.7 h during the observing window.


In [100]:
# that's a bit longer, at least

In [101]:
obs_range2 = [astro_set2, astro_rise2]

# Night 2 - North:

In [102]:
observing_table_n2 = observability_table(constraints2, mro, ntargets, time_range=obs_range2)

print(observing_table_n2)

      target name        ever observable ... fraction of time observable
------------------------ --------------- ... ---------------------------
               VI Cyg 12            True ...                         1.0
          GRS 79.29+0.46            True ...                         1.0
                 MWC 314            True ...                         1.0
           [OMN2000] LS1            True ...                         1.0
 2MASS J19325284+1742303            True ...                         1.0
2MASS J19443759+2419058c            True ...                         1.0
 2MASS J19011669+0355108            True ...                         0.8
                   MN101            True ...                         0.9
                   MN107            True ...                         1.0
                   P Cyg            True ...                         1.0
               AFGL 2298            True ...                         0.8


In [103]:
for i, target_table in enumerate(ntargets):

    if observing_table_n2['ever observable'][i]:
        name = observing_table_n2['target name'][i]
        obj_frac = observing_table_n2['fraction of time observable'][i]
        obj_time = obj_frac * observing_length2
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)

You can observe VI Cyg 12 for 4.68 h
You can observe GRS 79.29+0.46 for 4.68 h
You can observe MWC 314 for 4.68 h
You can observe [OMN2000] LS1 for 4.68 h
You can observe 2MASS J19325284+1742303 for 4.68 h
You can observe 2MASS J19443759+2419058c for 4.68 h
You can observe 2MASS J19011669+0355108 for 3.75 h
You can observe MN101 for 4.22 h
You can observe MN107 for 4.68 h
You can observe P Cyg for 4.68 h
You can observe AFGL 2298 for 3.75 h


# Now for Night 2 - South:

In [104]:
observing_table_s2 = observability_table(constraints2, mro, stargets, time_range=obs_range2)

print(observing_table_s2)

      target name       ever observable ... fraction of time observable
----------------------- --------------- ... ---------------------------
       GAL 024.73+00.69            True ...                         0.5
              HD 168625           False ...                         0.0
           GRS 25.5+0.2            True ...                         0.5
       GAL 026.47+00.02            True ...                         0.6
2MASS J18022233-2238002           False ...                         0.0
2MASS J18455593-0308297            True ...                         0.6
2MASS J18510295-0058242            True ...                         0.7
                   MN83            True ...                         0.4


In [105]:
for i, target_table in enumerate(stargets):

    if observing_table_s2['ever observable'][i]:
        name = observing_table_s2['target name'][i]
        obj_frac = observing_table_s2['fraction of time observable'][i]
        obj_time = obj_frac * observing_length2
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)

You can observe GAL 024.73+00.69 for 2.34 h
You can observe GRS 25.5+0.2 for 2.34 h
You can observe GAL 026.47+00.02 for 2.81 h
You can observe 2MASS J18455593-0308297 for 2.81 h
You can observe 2MASS J18510295-0058242 for 3.28 h
You can observe MN83 for 1.87 h
